In [2]:
#!pip list

In [3]:
#Dependancies
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras import backend as K

import matplotlib.pyplot as plt
import IPython
import numpy as np

In [4]:
#Organising Data from Cifar10 dataset
(X_train, Y_train), (X_test, Y_test) = datasets.cifar10.load_data()

#Takes the 2D array and converts to 1D
Y_train = Y_train.reshape(-1,)

#Divide each pixel value by 255 to normalise in a 0 to 1 range
X_train = X_train/255
X_test = X_test/255

#The classification word associated to each output Y of each input X
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

#Method to Plot the image with its classification
def plot_sample(X, Y, index):  
    plt.figure(figsize = (20,2)) #Changes image size
    plt.imshow(X[index], interpolation='bilinear') #plots the image
    plt.xlabel(classes[Y[index]]) #Adds a label for classification

In [11]:
X_train
#plot_sample(X_train, Y_train, 1)

array([[[[0.23137255, 0.24313725, 0.24705882],
         [0.16862745, 0.18039216, 0.17647059],
         [0.19607843, 0.18823529, 0.16862745],
         ...,
         [0.61960784, 0.51764706, 0.42352941],
         [0.59607843, 0.49019608, 0.4       ],
         [0.58039216, 0.48627451, 0.40392157]],

        [[0.0627451 , 0.07843137, 0.07843137],
         [0.        , 0.        , 0.        ],
         [0.07058824, 0.03137255, 0.        ],
         ...,
         [0.48235294, 0.34509804, 0.21568627],
         [0.46666667, 0.3254902 , 0.19607843],
         [0.47843137, 0.34117647, 0.22352941]],

        [[0.09803922, 0.09411765, 0.08235294],
         [0.0627451 , 0.02745098, 0.        ],
         [0.19215686, 0.10588235, 0.03137255],
         ...,
         [0.4627451 , 0.32941176, 0.19607843],
         [0.47058824, 0.32941176, 0.19607843],
         [0.42745098, 0.28627451, 0.16470588]],

        ...,

        [[0.81568627, 0.66666667, 0.37647059],
         [0.78823529, 0.6       , 0.13333333]

In [6]:
#Build model with layers
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [7]:
class PlotLossAccuracyCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_losses = []  # Train loss
        self.val_losses = []    # Validation loss
        self.train_accuracy = [] # Train accuracy
        self.val_accuracy = [] # Validation accuracy
        self.fig, (self.ax1, self.ax2) = plt.subplots(1, 2, figsize=(12,4))

    def on_epoch_end(self, epoch, logs={}):
        #Adjust x axis to start from 1 instead of 0
        epoch = epoch + 1

        self.train_losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.train_accuracy.append(logs.get('accuracy'))         # Train accuracy
        self.val_accuracy.append(logs.get('val_accuracy'))      # Validation accuracy

        # Plot loss in the left subplot for loss
        self.ax1.cla()
        self.ax1.plot(range(1, epoch+1), self.train_losses, label="train loss")
        self.ax1.plot(range(1, epoch+1), self.val_losses, label="validation loss")
        self.ax1.set_title('Loss')
        self.ax1.set_xlabel('Epoch')
        self.ax1.set_ylabel('Loss')
        self.ax1.legend()

        # Plot accuracy in the right subplot for accuracy
        self.ax2.cla()
        self.ax2.plot(range(1, epoch+1), self.train_accuracy, label="train accuracy")
        self.ax2.plot(range(1, epoch+1), self.val_accuracy, label="validation accuracy")
        self.ax2.set_title('Accuracy')
        self.ax2.set_xlabel('Epoch')
        self.ax2.set_ylabel('Accuracy')
        self.ax2.legend()

        IPython.display.clear_output(wait=True)
        IPython.display.display(self.fig)

In [8]:
#Compile the model
Myoptimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=Myoptimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

plot_loss_accuracy = PlotLossAccuracyCallback()

#Train the model
history = model.fit(X_train,
                    Y_train, 
                    epochs=30, 
                    validation_data=(X_test, Y_test),
                    callbacks=[plot_loss_accuracy]
                    )



Epoch 1/30
 427/1563 [=======>......................] - ETA: 33s - loss: 2.0660 - accuracy: 0.2414